[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/Digital-Finance-Introduction/blob/main/day_05/notebooks/NB12_DAO_Governance.ipynb)

# NB12: DAO Governance Simulation

**Topic:** 5.2 - Decentralized Governance & Risk

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand DAO Structure**: Explain the components and mechanics of Decentralized Autonomous Organizations
2. **Simulate Token-Weighted Voting**: Implement and analyze plutocratic voting systems
3. **Explore Quadratic Voting**: Understand alternative voting mechanisms that reduce wealth concentration
4. **Identify Governance Attacks**: Recognize vulnerabilities like whale dominance, vote buying, and flash loan attacks
5. **Analyze Delegation**: Implement liquid democracy and vote delegation
6. **Evaluate Security Mechanisms**: Understand time-locks, multi-sig, and other protective measures

## Section 1: Setup

We'll build a complete DAO governance simulation using only Python standard library and basic data science tools.

In [ ]:
# Install required packages
!pip install -q numpy pandas matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
from enum import Enum
from datetime import datetime, timedelta
import hashlib
import random
from collections import defaultdict

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("Libraries loaded successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## Section 2: What is a DAO?

A **Decentralized Autonomous Organization (DAO)** is an organization represented by rules encoded as a computer program that is transparent, controlled by organization members, and not influenced by a central government.

### Key Components of a DAO:

1. **Governance Token**: Represents voting power and/or ownership
2. **Proposal System**: Mechanism to submit changes for voting
3. **Voting Mechanism**: Rules for how votes are counted
4. **Execution Layer**: Smart contracts that execute approved proposals
5. **Treasury**: Shared funds controlled by governance

### Famous DAOs:

| DAO | Purpose | Token | Treasury (approx) |
|-----|---------|-------|-------------------|
| MakerDAO | Stablecoin governance | MKR | $5B+ |
| Uniswap | DEX governance | UNI | $3B+ |
| Compound | Lending protocol | COMP | $500M+ |
| Aave | Lending protocol | AAVE | $1B+ |
| Curve | DEX (stablecoins) | CRV | $2B+ |

In [ ]:
# Define core DAO data structures

class ProposalStatus(Enum):
    """Status of a governance proposal."""
    PENDING = "pending"
    ACTIVE = "active"
    SUCCEEDED = "succeeded"
    DEFEATED = "defeated"
    EXECUTED = "executed"
    EXPIRED = "expired"


@dataclass
class GovernanceToken:
    """Represents a governance token holder."""
    address: str
    balance: float
    delegated_to: Optional[str] = None
    delegated_from: List[str] = field(default_factory=list)
    
    def voting_power(self, all_holders: Dict[str, 'GovernanceToken']) -> float:
        """Calculate total voting power including delegations."""
        if self.delegated_to is not None:
            return 0  # Power delegated away
        
        power = self.balance
        for delegator_addr in self.delegated_from:
            if delegator_addr in all_holders:
                power += all_holders[delegator_addr].balance
        return power


@dataclass
class Proposal:
    """A governance proposal."""
    id: int
    title: str
    description: str
    proposer: str
    created_at: datetime
    voting_ends: datetime
    votes_for: float = 0
    votes_against: float = 0
    votes_abstain: float = 0
    voters: Dict[str, str] = field(default_factory=dict)  # address -> vote
    status: ProposalStatus = ProposalStatus.PENDING
    quorum_required: float = 0.04  # 4% of total supply
    
    @property
    def total_votes(self) -> float:
        return self.votes_for + self.votes_against + self.votes_abstain
    
    @property
    def approval_rate(self) -> float:
        if self.votes_for + self.votes_against == 0:
            return 0
        return self.votes_for / (self.votes_for + self.votes_against)


print("DAO data structures defined:")
print("  - ProposalStatus: Enum for proposal lifecycle")
print("  - GovernanceToken: Token holder with delegation support")
print("  - Proposal: Governance proposal with voting")

In [ ]:
class SimpleDAO:
    """
    A simplified DAO implementation for educational purposes.
    Demonstrates core governance mechanics.
    """
    
    def __init__(self, name: str, token_symbol: str, total_supply: float):
        self.name = name
        self.token_symbol = token_symbol
        self.total_supply = total_supply
        self.holders: Dict[str, GovernanceToken] = {}
        self.proposals: Dict[int, Proposal] = {}
        self.treasury: float = 0
        self.next_proposal_id = 1
        
        # Governance parameters
        self.proposal_threshold = 0.01  # 1% of supply to propose
        self.quorum = 0.04  # 4% participation required
        self.voting_period_days = 7
        self.timelock_days = 2
    
    def add_holder(self, address: str, balance: float) -> None:
        """Add a token holder to the DAO."""
        self.holders[address] = GovernanceToken(address=address, balance=balance)
    
    def get_voting_power(self, address: str) -> float:
        """Get voting power for an address (including delegations)."""
        if address not in self.holders:
            return 0
        return self.holders[address].voting_power(self.holders)
    
    def can_propose(self, address: str) -> bool:
        """Check if address has enough tokens to create proposal."""
        power = self.get_voting_power(address)
        return power >= self.total_supply * self.proposal_threshold
    
    def create_proposal(self, proposer: str, title: str, description: str) -> Optional[Proposal]:
        """Create a new governance proposal."""
        if not self.can_propose(proposer):
            print(f"Error: {proposer} needs {self.proposal_threshold*100}% of tokens to propose")
            return None
        
        now = datetime.now()
        proposal = Proposal(
            id=self.next_proposal_id,
            title=title,
            description=description,
            proposer=proposer,
            created_at=now,
            voting_ends=now + timedelta(days=self.voting_period_days),
            quorum_required=self.quorum,
            status=ProposalStatus.ACTIVE
        )
        
        self.proposals[proposal.id] = proposal
        self.next_proposal_id += 1
        return proposal
    
    def vote(self, proposal_id: int, voter: str, support: str) -> bool:
        """
        Cast a vote on a proposal.
        support: 'for', 'against', or 'abstain'
        """
        if proposal_id not in self.proposals:
            print(f"Proposal {proposal_id} not found")
            return False
        
        proposal = self.proposals[proposal_id]
        
        if proposal.status != ProposalStatus.ACTIVE:
            print(f"Proposal is not active (status: {proposal.status.value})")
            return False
        
        if voter in proposal.voters:
            print(f"{voter} has already voted")
            return False
        
        voting_power = self.get_voting_power(voter)
        if voting_power == 0:
            print(f"{voter} has no voting power")
            return False
        
        proposal.voters[voter] = support
        
        if support == 'for':
            proposal.votes_for += voting_power
        elif support == 'against':
            proposal.votes_against += voting_power
        else:
            proposal.votes_abstain += voting_power
        
        return True
    
    def finalize_proposal(self, proposal_id: int) -> ProposalStatus:
        """Finalize a proposal after voting period ends."""
        proposal = self.proposals[proposal_id]
        
        # Check quorum
        participation = proposal.total_votes / self.total_supply
        quorum_met = participation >= proposal.quorum_required
        
        # Check approval (simple majority)
        approved = proposal.votes_for > proposal.votes_against
        
        if quorum_met and approved:
            proposal.status = ProposalStatus.SUCCEEDED
        else:
            proposal.status = ProposalStatus.DEFEATED
        
        return proposal.status
    
    def get_token_distribution(self) -> pd.DataFrame:
        """Get token distribution as DataFrame."""
        data = []
        for addr, holder in self.holders.items():
            data.append({
                'address': addr,
                'balance': holder.balance,
                'percentage': holder.balance / self.total_supply * 100,
                'voting_power': self.get_voting_power(addr),
                'delegated_to': holder.delegated_to
            })
        return pd.DataFrame(data).sort_values('balance', ascending=False)


# Create a sample DAO
dao = SimpleDAO("ExampleDAO", "EXDAO", total_supply=1_000_000)
print(f"Created {dao.name} with {dao.total_supply:,} {dao.token_symbol} tokens")
print(f"\nGovernance Parameters:")
print(f"  Proposal threshold: {dao.proposal_threshold*100}% ({dao.total_supply * dao.proposal_threshold:,.0f} tokens)")
print(f"  Quorum required: {dao.quorum*100}% ({dao.total_supply * dao.quorum:,.0f} tokens)")
print(f"  Voting period: {dao.voting_period_days} days")

## Section 3: Token-Weighted Voting Simulation

**Token-weighted voting** (also called plutocratic voting) is the most common governance mechanism in DAOs. Each token equals one vote.

**Advantages:**
- Simple and easy to understand
- Aligns voting power with economic stake
- Resistant to Sybil attacks (creating fake identities)

**Disadvantages:**
- Wealth concentration leads to power concentration
- Whales can dominate decisions
- Small holders have negligible influence

In [ ]:
def generate_realistic_distribution(total_supply: float, num_holders: int) -> Dict[str, float]:
    """
    Generate a realistic token distribution following power law.
    This mimics real-world crypto token distributions.
    """
    # Power law distribution (Pareto-like)
    # Top holders have disproportionate share
    
    holders = {}
    
    # Team/Foundation: 20%
    holders['team_treasury'] = total_supply * 0.15
    holders['foundation'] = total_supply * 0.05
    
    # Whales (5 addresses): 25%
    whale_share = total_supply * 0.25
    whale_amounts = [0.10, 0.06, 0.04, 0.03, 0.02]  # Percentages
    for i, pct in enumerate(whale_amounts):
        holders[f'whale_{i+1}'] = total_supply * pct
    
    # Medium holders (20 addresses): 30%
    medium_share = total_supply * 0.30
    medium_amounts = np.random.pareto(2, 20)
    medium_amounts = medium_amounts / medium_amounts.sum() * medium_share
    for i, amount in enumerate(medium_amounts):
        holders[f'holder_{i+1}'] = amount
    
    # Retail (remaining): 20%
    retail_share = total_supply * 0.20
    num_retail = num_holders - len(holders)
    retail_amounts = np.random.exponential(1, num_retail)
    retail_amounts = retail_amounts / retail_amounts.sum() * retail_share
    for i, amount in enumerate(retail_amounts):
        holders[f'retail_{i+1}'] = amount
    
    return holders


# Generate distribution and populate DAO
distribution = generate_realistic_distribution(dao.total_supply, 100)

for address, balance in distribution.items():
    dao.add_holder(address, balance)

# Display distribution
df_dist = dao.get_token_distribution()
print("Token Distribution (Top 15 holders):")
print("=" * 70)
print(df_dist.head(15).to_string(index=False))

# Calculate concentration metrics
top_10_pct = df_dist.head(10)['percentage'].sum()
print(f"\nConcentration Metrics:")
print(f"  Top 10 holders control: {top_10_pct:.1f}%")
print(f"  Top 1 holder controls: {df_dist.iloc[0]['percentage']:.1f}%")
print(f"  Median holding: {df_dist['balance'].median():,.0f} tokens ({df_dist['balance'].median()/dao.total_supply*100:.2f}%)")

In [ ]:
# Visualize distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Top holders bar chart
top_20 = df_dist.head(20)
colors = ['#e74c3c' if 'whale' in addr or 'team' in addr or 'foundation' in addr 
          else '#3498db' for addr in top_20['address']]
axes[0].barh(range(len(top_20)), top_20['percentage'], color=colors)
axes[0].set_yticks(range(len(top_20)))
axes[0].set_yticklabels(top_20['address'], fontsize=8)
axes[0].set_xlabel('Percentage of Total Supply')
axes[0].set_title('Top 20 Token Holders')
axes[0].invert_yaxis()

# Right: Cumulative distribution (Lorenz curve style)
cumulative = df_dist['percentage'].cumsum()
axes[1].plot(range(1, len(cumulative)+1), cumulative, 'b-', linewidth=2, label='Actual')
axes[1].plot([0, len(cumulative)], [0, 100], 'r--', label='Perfect Equality')
axes[1].fill_between(range(1, len(cumulative)+1), cumulative, alpha=0.3)
axes[1].set_xlabel('Number of Holders (ranked by size)')
axes[1].set_ylabel('Cumulative % of Supply')
axes[1].set_title('Token Concentration Curve')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight: The area between the curves represents inequality.")
print("Most DAOs have highly concentrated token distributions.")

In [ ]:
# Simulate a governance vote
print("=" * 70)
print("GOVERNANCE VOTE SIMULATION")
print("=" * 70)

# Create a proposal
proposal = dao.create_proposal(
    proposer='whale_1',  # Has enough tokens to propose
    title="Increase Treasury Allocation",
    description="Allocate 5% of treasury to development grants"
)

print(f"\nProposal #{proposal.id}: {proposal.title}")
print(f"Proposer: whale_1 (voting power: {dao.get_voting_power('whale_1'):,.0f} tokens)")

# Simulate voting behavior
# Whales tend to vote, retail often doesn't
vote_probability = {
    'whale': 0.9,      # Whales almost always vote
    'team': 0.8,       # Team usually votes
    'holder': 0.5,     # Medium holders sometimes vote
    'retail': 0.1,     # Retail rarely votes
    'foundation': 0.3  # Foundation sometimes abstains
}

votes_cast = {'for': 0, 'against': 0, 'abstain': 0}

for address in dao.holders:
    # Determine vote probability based on holder type
    holder_type = address.split('_')[0]
    prob = vote_probability.get(holder_type, 0.3)
    
    if random.random() < prob:
        # Determine vote choice (whales more likely to support their own proposal)
        if 'whale' in address:
            vote_choice = 'for' if random.random() < 0.8 else 'against'
        elif 'team' in address or 'foundation' in address:
            vote_choice = random.choices(['for', 'against', 'abstain'], weights=[0.5, 0.3, 0.2])[0]
        else:
            vote_choice = random.choices(['for', 'against', 'abstain'], weights=[0.4, 0.4, 0.2])[0]
        
        if dao.vote(proposal.id, address, vote_choice):
            votes_cast[vote_choice] += 1

# Display results
print(f"\nVoting Results:")
print(f"  Votes FOR:     {proposal.votes_for:>12,.0f} tokens ({proposal.votes_for/dao.total_supply*100:.2f}%)")
print(f"  Votes AGAINST: {proposal.votes_against:>12,.0f} tokens ({proposal.votes_against/dao.total_supply*100:.2f}%)")
print(f"  Votes ABSTAIN: {proposal.votes_abstain:>12,.0f} tokens ({proposal.votes_abstain/dao.total_supply*100:.2f}%)")
print(f"\n  Participation: {proposal.total_votes/dao.total_supply*100:.2f}%")
print(f"  Quorum required: {dao.quorum*100}%")
print(f"  Quorum met: {'Yes' if proposal.total_votes/dao.total_supply >= dao.quorum else 'No'}")
print(f"\n  Approval rate: {proposal.approval_rate*100:.1f}%")

# Finalize
final_status = dao.finalize_proposal(proposal.id)
print(f"\nFinal Status: {final_status.value.upper()}")

## Section 4: Quadratic Voting

**Quadratic Voting** is an alternative mechanism designed to reduce the influence of wealthy participants.

**How it works:**
- Voting power = sqrt(tokens)
- 100 tokens = 10 votes
- 10,000 tokens = 100 votes (not 100x more!)

**Benefits:**
- Reduces whale dominance
- Gives more voice to small holders
- Encourages broader participation

**Challenges:**
- Vulnerable to Sybil attacks (splitting tokens across wallets)
- Requires identity verification to be truly effective

In [ ]:
def compare_voting_systems(holders: Dict[str, float]) -> pd.DataFrame:
    """
    Compare token-weighted vs quadratic voting power.
    """
    total_tokens = sum(holders.values())
    
    data = []
    for address, balance in holders.items():
        linear_power = balance
        quadratic_power = np.sqrt(balance)
        
        data.append({
            'address': address,
            'tokens': balance,
            'linear_power': linear_power,
            'quadratic_power': quadratic_power
        })
    
    df = pd.DataFrame(data)
    
    # Normalize to percentages
    df['linear_pct'] = df['linear_power'] / df['linear_power'].sum() * 100
    df['quadratic_pct'] = df['quadratic_power'] / df['quadratic_power'].sum() * 100
    
    return df.sort_values('tokens', ascending=False)


# Compare systems
comparison = compare_voting_systems(distribution)

print("Voting Power Comparison: Linear vs Quadratic")
print("=" * 80)
print(f"{'Address':<20} {'Tokens':>12} {'Linear %':>12} {'Quadratic %':>12} {'Reduction':>12}")
print("-" * 80)

for _, row in comparison.head(15).iterrows():
    reduction = (1 - row['quadratic_pct']/row['linear_pct']) * 100 if row['linear_pct'] > 0 else 0
    print(f"{row['address']:<20} {row['tokens']:>12,.0f} {row['linear_pct']:>11.2f}% {row['quadratic_pct']:>11.2f}% {reduction:>11.1f}%")

# Summary statistics
top_5_linear = comparison.head(5)['linear_pct'].sum()
top_5_quadratic = comparison.head(5)['quadratic_pct'].sum()

print("\n" + "=" * 80)
print(f"Top 5 holders' combined power:")
print(f"  Linear voting:    {top_5_linear:.1f}%")
print(f"  Quadratic voting: {top_5_quadratic:.1f}%")
print(f"  Power reduction:  {top_5_linear - top_5_quadratic:.1f} percentage points")

In [ ]:
# Visualize the difference
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

top_10 = comparison.head(10)

# Left: Side-by-side comparison
x = np.arange(len(top_10))
width = 0.35

axes[0].bar(x - width/2, top_10['linear_pct'], width, label='Linear (1 token = 1 vote)', color='#e74c3c')
axes[0].bar(x + width/2, top_10['quadratic_pct'], width, label='Quadratic (sqrt tokens)', color='#27ae60')
axes[0].set_xlabel('Holder')
axes[0].set_ylabel('Voting Power (%)')
axes[0].set_title('Voting Power: Linear vs Quadratic')
axes[0].set_xticks(x)
axes[0].set_xticklabels([addr[:10] for addr in top_10['address']], rotation=45, ha='right')
axes[0].legend()

# Right: Power curve comparison
token_range = np.logspace(1, 6, 100)  # 10 to 1,000,000 tokens
linear_power = token_range
quadratic_power = np.sqrt(token_range)

# Normalize
linear_normalized = linear_power / linear_power.max() * 100
quadratic_normalized = quadratic_power / quadratic_power.max() * 100

axes[1].semilogx(token_range, linear_normalized, 'r-', label='Linear', linewidth=2)
axes[1].semilogx(token_range, quadratic_normalized, 'g-', label='Quadratic', linewidth=2)
axes[1].set_xlabel('Token Holdings')
axes[1].set_ylabel('Relative Voting Power (%)')
axes[1].set_title('Voting Power Curves')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight: Quadratic voting significantly flattens the power curve,")
print("giving smaller holders proportionally more influence.")

## Section 5: Governance Attack Vectors

DAOs face several attack vectors that can compromise governance:

### 1. Whale Dominance
- Large holders can unilaterally pass proposals
- May not align with community interests

### 2. Vote Buying
- Bribing token holders to vote a certain way
- Can be done openly via DeFi protocols

### 3. Flash Loan Governance Attacks
- Borrow tokens, vote, return tokens - all in one transaction
- Especially dangerous if voting doesn't require token lock-up

### 4. Low Participation Exploitation
- With low turnout, small amounts can swing votes
- Attackers time proposals for low-activity periods

In [ ]:
def simulate_whale_attack(dao: SimpleDAO, whale_address: str) -> Dict:
    """
    Simulate a whale attempting to pass a self-serving proposal.
    """
    print("=" * 70)
    print("WHALE DOMINANCE ATTACK SIMULATION")
    print("=" * 70)
    
    whale_power = dao.get_voting_power(whale_address)
    whale_pct = whale_power / dao.total_supply * 100
    
    print(f"\nAttacker: {whale_address}")
    print(f"Voting Power: {whale_power:,.0f} tokens ({whale_pct:.2f}%)")
    
    # Can whale pass proposal alone?
    can_pass_alone = whale_pct > 50
    
    # With typical participation (let's say 15%)
    typical_participation = 0.15
    expected_other_votes = dao.total_supply * typical_participation - whale_power
    
    # If whale votes and others split 50/50
    others_for = expected_other_votes * 0.5
    others_against = expected_other_votes * 0.5
    
    whale_wins = (whale_power + others_for) > others_against
    
    print(f"\nScenario Analysis:")
    print(f"  Can pass alone (>50%): {'Yes' if can_pass_alone else 'No'}")
    print(f"\n  With {typical_participation*100:.0f}% typical participation:")
    print(f"    Expected votes from others: {expected_other_votes:,.0f}")
    print(f"    If others split 50/50, whale wins: {'Yes' if whale_wins else 'No'}")
    
    # Calculate threshold - what % opposition needed to defeat whale?
    # whale_power < others_against
    # whale_power < expected_other_votes * opposition_rate
    # opposition_rate > whale_power / expected_other_votes
    
    if expected_other_votes > 0:
        opposition_needed = whale_power / expected_other_votes * 100
        print(f"\n  To defeat whale's proposal:")
        print(f"    {opposition_needed:.1f}% of other voters must vote AGAINST")
    
    return {
        'whale_power': whale_power,
        'whale_percentage': whale_pct,
        'can_pass_alone': can_pass_alone,
        'wins_with_typical_participation': whale_wins
    }

# Simulate attack
attack_result = simulate_whale_attack(dao, 'whale_1')

In [ ]:
def simulate_flash_loan_attack(dao: SimpleDAO, attacker: str, loan_amount: float) -> Dict:
    """
    Simulate a flash loan governance attack.
    
    In a flash loan attack:
    1. Attacker borrows tokens (no collateral needed)
    2. Votes on proposal with borrowed tokens
    3. Returns tokens in same transaction
    
    This only works if the DAO doesn't have proper safeguards.
    """
    print("\n" + "=" * 70)
    print("FLASH LOAN GOVERNANCE ATTACK SIMULATION")
    print("=" * 70)
    
    # Attacker's original balance
    original_balance = dao.holders.get(attacker, GovernanceToken(attacker, 0)).balance
    
    print(f"\nAttacker: {attacker}")
    print(f"Original Balance: {original_balance:,.0f} tokens")
    print(f"Flash Loan Amount: {loan_amount:,.0f} tokens")
    
    # Simulate temporary balance increase
    temp_balance = original_balance + loan_amount
    temp_voting_power_pct = temp_balance / dao.total_supply * 100
    
    print(f"\nDuring Attack:")
    print(f"  Temporary Balance: {temp_balance:,.0f} tokens")
    print(f"  Temporary Voting Power: {temp_voting_power_pct:.2f}%")
    
    # Can they swing a vote?
    can_meet_quorum = temp_balance >= dao.total_supply * dao.quorum
    can_pass_alone = temp_voting_power_pct > 50
    
    print(f"\nAttack Potential:")
    print(f"  Can meet quorum alone: {'Yes' if can_meet_quorum else 'No'}")
    print(f"  Can pass proposal alone: {'Yes' if can_pass_alone else 'No'}")
    
    # Cost analysis (flash loan fees are typically 0.09%)
    flash_loan_fee_rate = 0.0009
    attack_cost = loan_amount * flash_loan_fee_rate
    
    print(f"\nAttack Economics:")
    print(f"  Flash Loan Fee (0.09%): {attack_cost:,.0f} tokens")
    print(f"  Gas Costs: ~0.1 ETH")
    
    # Defenses
    print(f"\nDefenses Against This Attack:")
    print(f"  1. Snapshot voting power BEFORE proposal creation")
    print(f"  2. Require tokens locked during voting period")
    print(f"  3. Time-weighted voting (hold tokens for X days)")
    print(f"  4. Voting delay after token acquisition")
    
    return {
        'original_balance': original_balance,
        'loan_amount': loan_amount,
        'temp_power_pct': temp_voting_power_pct,
        'can_pass_alone': can_pass_alone,
        'attack_cost': attack_cost
    }

# Simulate attack with 200,000 token flash loan
flash_attack = simulate_flash_loan_attack(dao, 'retail_1', 200_000)

In [ ]:
def simulate_vote_buying(dao: SimpleDAO, budget: float, token_price: float = 10.0) -> Dict:
    """
    Simulate a vote buying attack.
    
    Attacker offers bribes to token holders to vote a certain way.
    """
    print("\n" + "=" * 70)
    print("VOTE BUYING ATTACK SIMULATION")
    print("=" * 70)
    
    print(f"\nAttacker Budget: ${budget:,.0f}")
    print(f"Token Price: ${token_price:.2f}")
    
    # Calculate bribe economics
    # Rational voter will sell vote if bribe > expected value of voting preference
    # Assume voters value their vote at 1-5% of token value
    
    bribe_rates = [0.01, 0.02, 0.05]  # 1%, 2%, 5% of token value
    
    print(f"\nBribe Analysis:")
    print(f"{'Bribe Rate':<15} {'Cost/Vote':>12} {'Votes Buyable':>15} {'% of Supply':>12}")
    print("-" * 60)
    
    results = []
    for rate in bribe_rates:
        cost_per_vote = token_price * rate
        votes_buyable = budget / cost_per_vote
        pct_of_supply = votes_buyable / dao.total_supply * 100
        
        print(f"{rate*100:.0f}%{'':<12} ${cost_per_vote:>10.2f} {votes_buyable:>14,.0f} {pct_of_supply:>11.2f}%")
        
        results.append({
            'bribe_rate': rate,
            'cost_per_vote': cost_per_vote,
            'votes_buyable': votes_buyable,
            'pct_of_supply': pct_of_supply
        })
    
    # Calculate cost to swing typical vote
    # Assume 15% participation, 50/50 split
    participation = 0.15
    total_votes = dao.total_supply * participation
    votes_needed_to_swing = total_votes * 0.01  # Need 1% swing to win
    
    print(f"\nCost to Swing Close Vote (15% participation):")
    print(f"  Votes needed to swing: {votes_needed_to_swing:,.0f}")
    
    for rate in bribe_rates:
        cost = votes_needed_to_swing * token_price * rate
        print(f"  At {rate*100:.0f}% bribe rate: ${cost:,.0f}")
    
    print(f"\nDefenses:")
    print(f"  1. Secret/commit-reveal voting")
    print(f"  2. Conviction voting (longer stakes = more power)")
    print(f"  3. Reputation-weighted voting")
    print(f"  4. Quadratic voting (makes large bribes expensive)")
    
    return results

# Simulate with $100,000 budget
bribe_results = simulate_vote_buying(dao, budget=100_000)

## Section 6: Delegation Mechanisms

**Vote Delegation** (Liquid Democracy) allows token holders to delegate their voting power to trusted representatives.

**Benefits:**
- Enables participation without active voting
- Allows expertise to be leveraged
- Can increase effective participation rates

**Risks:**
- Power concentration in delegates
- Delegates may not represent delegators' interests
- Creates new attack surface (bribing delegates)

In [ ]:
class DelegatingDAO(SimpleDAO):
    """DAO with full delegation support."""
    
    def delegate(self, from_address: str, to_address: str) -> bool:
        """Delegate voting power to another address."""
        if from_address not in self.holders:
            print(f"{from_address} is not a token holder")
            return False
        
        if to_address not in self.holders:
            print(f"{to_address} is not a token holder")
            return False
        
        if from_address == to_address:
            print("Cannot delegate to self")
            return False
        
        delegator = self.holders[from_address]
        delegate = self.holders[to_address]
        
        # Remove from previous delegate if any
        if delegator.delegated_to is not None:
            old_delegate = self.holders[delegator.delegated_to]
            if from_address in old_delegate.delegated_from:
                old_delegate.delegated_from.remove(from_address)
        
        # Set up new delegation
        delegator.delegated_to = to_address
        delegate.delegated_from.append(from_address)
        
        return True
    
    def undelegate(self, address: str) -> bool:
        """Remove delegation and reclaim voting power."""
        if address not in self.holders:
            return False
        
        holder = self.holders[address]
        
        if holder.delegated_to is None:
            print(f"{address} has not delegated")
            return False
        
        delegate = self.holders[holder.delegated_to]
        if address in delegate.delegated_from:
            delegate.delegated_from.remove(address)
        
        holder.delegated_to = None
        return True
    
    def get_delegation_stats(self) -> Dict:
        """Get statistics about delegations."""
        delegators = [h for h in self.holders.values() if h.delegated_to is not None]
        delegates = [h for h in self.holders.values() if len(h.delegated_from) > 0]
        
        total_delegated = sum(h.balance for h in delegators)
        
        return {
            'num_delegators': len(delegators),
            'num_delegates': len(delegates),
            'total_delegated': total_delegated,
            'pct_delegated': total_delegated / self.total_supply * 100
        }


# Create DAO with delegation
del_dao = DelegatingDAO("DelegateDAO", "DDAO", total_supply=1_000_000)

# Add holders
for address, balance in distribution.items():
    del_dao.add_holder(address, balance)

print("Created DAO with delegation support")
print(f"Total holders: {len(del_dao.holders)}")

In [ ]:
# Simulate realistic delegation patterns
print("Simulating Delegation Patterns")
print("=" * 70)

# Popular delegates (would be well-known community members)
popular_delegates = ['whale_1', 'whale_2', 'holder_1', 'holder_2', 'holder_3']

# Retail holders often delegate to known figures
for address in del_dao.holders:
    if 'retail' in address:
        # 40% of retail delegates
        if random.random() < 0.4:
            delegate = random.choice(popular_delegates)
            del_dao.delegate(address, delegate)
    elif 'holder' in address and address not in popular_delegates:
        # 20% of medium holders delegate
        if random.random() < 0.2:
            delegate = random.choice(popular_delegates[:3])  # Prefer top delegates
            del_dao.delegate(address, delegate)

# Get stats
stats = del_dao.get_delegation_stats()
print(f"\nDelegation Statistics:")
print(f"  Delegators: {stats['num_delegators']}")
print(f"  Delegates receiving votes: {stats['num_delegates']}")
print(f"  Total delegated: {stats['total_delegated']:,.0f} tokens ({stats['pct_delegated']:.1f}%)")

# Show delegate power
print(f"\nTop Delegates by Voting Power:")
print(f"{'Delegate':<20} {'Own Tokens':>12} {'Delegated':>12} {'Total Power':>12} {'% Supply':>10}")
print("-" * 70)

delegate_powers = []
for delegate in popular_delegates:
    holder = del_dao.holders[delegate]
    own_tokens = holder.balance
    delegated = sum(del_dao.holders[d].balance for d in holder.delegated_from)
    total_power = del_dao.get_voting_power(delegate)
    pct = total_power / del_dao.total_supply * 100
    
    delegate_powers.append((delegate, own_tokens, delegated, total_power, pct))

delegate_powers.sort(key=lambda x: x[3], reverse=True)

for delegate, own, delegated, total, pct in delegate_powers:
    print(f"{delegate:<20} {own:>12,.0f} {delegated:>12,.0f} {total:>12,.0f} {pct:>9.2f}%")

In [ ]:
# Visualize delegation network
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Power distribution before/after delegation
before_delegation = [del_dao.holders[d].balance for d in popular_delegates]
after_delegation = [del_dao.get_voting_power(d) for d in popular_delegates]

x = np.arange(len(popular_delegates))
width = 0.35

axes[0].bar(x - width/2, before_delegation, width, label='Own Tokens', color='#3498db')
axes[0].bar(x + width/2, after_delegation, width, label='With Delegations', color='#e74c3c')
axes[0].set_xlabel('Delegate')
axes[0].set_ylabel('Voting Power')
axes[0].set_title('Delegate Power: Own vs Delegated')
axes[0].set_xticks(x)
axes[0].set_xticklabels(popular_delegates, rotation=45, ha='right')
axes[0].legend()
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1000:.0f}K'))

# Right: Delegation concentration
all_powers = [(addr, del_dao.get_voting_power(addr)) for addr in del_dao.holders]
all_powers.sort(key=lambda x: x[1], reverse=True)
top_powers = [p[1] for p in all_powers[:20] if p[1] > 0]
labels = [p[0][:10] for p in all_powers[:20] if p[1] > 0]

axes[1].pie(top_powers[:10], labels=labels[:10], autopct='%1.1f%%', startangle=90)
axes[1].set_title('Voting Power Concentration (Top 10)')

plt.tight_layout()
plt.show()

print("\nKey Insight: Delegation can dramatically concentrate voting power.")
print("Popular delegates gain significant influence through trust.")

## Section 7: Time-locks and Multi-sig

**Security mechanisms** protect DAOs from rapid, malicious changes:

### Time-locks
- Delay between proposal approval and execution
- Gives community time to react to malicious proposals
- Typical delay: 24-72 hours

### Multi-signature (Multi-sig)
- Requires multiple parties to approve actions
- Common patterns: 2-of-3, 3-of-5, 4-of-7
- Prevents single point of failure

In [ ]:
@dataclass
class TimelockProposal:
    """A proposal with timelock protection."""
    proposal: Proposal
    queued_at: Optional[datetime] = None
    executable_at: Optional[datetime] = None
    executed_at: Optional[datetime] = None
    cancelled: bool = False


class TimelockGovernor:
    """
    Governance with timelock protection.
    Successful proposals must wait before execution.
    """
    
    def __init__(self, delay_hours: int = 48):
        self.delay_hours = delay_hours
        self.queued_proposals: Dict[int, TimelockProposal] = {}
    
    def queue_proposal(self, proposal: Proposal) -> Optional[TimelockProposal]:
        """Queue a successful proposal for execution."""
        if proposal.status != ProposalStatus.SUCCEEDED:
            print(f"Only succeeded proposals can be queued (status: {proposal.status.value})")
            return None
        
        now = datetime.now()
        timelock = TimelockProposal(
            proposal=proposal,
            queued_at=now,
            executable_at=now + timedelta(hours=self.delay_hours)
        )
        
        self.queued_proposals[proposal.id] = timelock
        return timelock
    
    def can_execute(self, proposal_id: int) -> Tuple[bool, str]:
        """Check if a proposal can be executed."""
        if proposal_id not in self.queued_proposals:
            return False, "Proposal not queued"
        
        timelock = self.queued_proposals[proposal_id]
        
        if timelock.cancelled:
            return False, "Proposal was cancelled"
        
        if timelock.executed_at is not None:
            return False, "Proposal already executed"
        
        now = datetime.now()
        if now < timelock.executable_at:
            time_remaining = timelock.executable_at - now
            return False, f"Timelock active: {time_remaining.total_seconds()/3600:.1f} hours remaining"
        
        return True, "Ready to execute"
    
    def cancel_proposal(self, proposal_id: int, canceller: str) -> bool:
        """Cancel a queued proposal (guardian function)."""
        if proposal_id not in self.queued_proposals:
            return False
        
        self.queued_proposals[proposal_id].cancelled = True
        print(f"Proposal {proposal_id} cancelled by {canceller}")
        return True


# Demonstrate timelock
print("Timelock Governance Demonstration")
print("=" * 70)

governor = TimelockGovernor(delay_hours=48)

# Create a mock succeeded proposal
mock_proposal = Proposal(
    id=1,
    title="Transfer 10,000 tokens from treasury",
    description="Fund development team",
    proposer="whale_1",
    created_at=datetime.now() - timedelta(days=7),
    voting_ends=datetime.now() - timedelta(days=1),
    votes_for=150000,
    votes_against=50000,
    status=ProposalStatus.SUCCEEDED
)

print(f"\nProposal: {mock_proposal.title}")
print(f"Status: {mock_proposal.status.value}")
print(f"Approval: {mock_proposal.approval_rate*100:.1f}%")

# Queue it
timelock = governor.queue_proposal(mock_proposal)
print(f"\nQueued at: {timelock.queued_at.strftime('%Y-%m-%d %H:%M')}")
print(f"Executable at: {timelock.executable_at.strftime('%Y-%m-%d %H:%M')}")
print(f"Delay: {governor.delay_hours} hours")

# Check if executable
can_exec, reason = governor.can_execute(1)
print(f"\nCan execute now: {can_exec}")
print(f"Reason: {reason}")

print(f"\nTimelock Purpose:")
print(f"  - Gives community time to review and react")
print(f"  - Malicious proposals can be cancelled by guardians")
print(f"  - Users can exit before harmful changes take effect")

In [ ]:
class MultiSigWallet:
    """
    Multi-signature wallet for DAO treasury.
    Requires M of N signers to approve transactions.
    """
    
    def __init__(self, signers: List[str], required: int):
        if required > len(signers):
            raise ValueError("Required signatures cannot exceed number of signers")
        if required < 1:
            raise ValueError("At least 1 signature required")
        
        self.signers = set(signers)
        self.required = required
        self.pending_transactions: Dict[int, Dict] = {}
        self.next_tx_id = 1
        self.balance = 0
    
    def deposit(self, amount: float) -> None:
        """Deposit funds to the multisig."""
        self.balance += amount
    
    def propose_transaction(self, proposer: str, to: str, amount: float, 
                           description: str) -> Optional[int]:
        """Propose a new transaction (must be a signer)."""
        if proposer not in self.signers:
            print(f"{proposer} is not a signer")
            return None
        
        if amount > self.balance:
            print(f"Insufficient balance: {self.balance:,.0f} < {amount:,.0f}")
            return None
        
        tx_id = self.next_tx_id
        self.pending_transactions[tx_id] = {
            'to': to,
            'amount': amount,
            'description': description,
            'proposer': proposer,
            'signatures': {proposer},  # Proposer auto-signs
            'executed': False
        }
        self.next_tx_id += 1
        return tx_id
    
    def sign(self, tx_id: int, signer: str) -> bool:
        """Sign a pending transaction."""
        if signer not in self.signers:
            print(f"{signer} is not a signer")
            return False
        
        if tx_id not in self.pending_transactions:
            print(f"Transaction {tx_id} not found")
            return False
        
        tx = self.pending_transactions[tx_id]
        if tx['executed']:
            print(f"Transaction {tx_id} already executed")
            return False
        
        tx['signatures'].add(signer)
        return True
    
    def execute(self, tx_id: int) -> bool:
        """Execute a transaction if enough signatures."""
        if tx_id not in self.pending_transactions:
            return False
        
        tx = self.pending_transactions[tx_id]
        
        if tx['executed']:
            print("Already executed")
            return False
        
        if len(tx['signatures']) < self.required:
            print(f"Need {self.required} signatures, have {len(tx['signatures'])}")
            return False
        
        if tx['amount'] > self.balance:
            print("Insufficient balance")
            return False
        
        self.balance -= tx['amount']
        tx['executed'] = True
        print(f"Executed: Sent {tx['amount']:,.0f} to {tx['to']}")
        return True


# Demonstrate multi-sig
print("\nMulti-Signature Wallet Demonstration")
print("=" * 70)

# Create 3-of-5 multisig
signers = ['alice', 'bob', 'charlie', 'david', 'eve']
multisig = MultiSigWallet(signers, required=3)
multisig.deposit(100_000)  # 100k tokens

print(f"\nMultisig Configuration: 3-of-5")
print(f"Signers: {', '.join(signers)}")
print(f"Balance: {multisig.balance:,} tokens")

# Propose a transaction
tx_id = multisig.propose_transaction(
    proposer='alice',
    to='dev_team',
    amount=25_000,
    description='Q1 development grant'
)

print(f"\nTransaction {tx_id} proposed by alice")
print(f"  Amount: 25,000 tokens")
print(f"  To: dev_team")
print(f"  Signatures: {len(multisig.pending_transactions[tx_id]['signatures'])}/{multisig.required}")

# Try to execute (not enough sigs)
print(f"\nAttempting execution...")
multisig.execute(tx_id)

# Get more signatures
multisig.sign(tx_id, 'bob')
print(f"Bob signed: {len(multisig.pending_transactions[tx_id]['signatures'])}/{multisig.required}")

multisig.sign(tx_id, 'charlie')
print(f"Charlie signed: {len(multisig.pending_transactions[tx_id]['signatures'])}/{multisig.required}")

# Now execute
print(f"\nAttempting execution with enough signatures...")
multisig.execute(tx_id)

print(f"\nRemaining balance: {multisig.balance:,} tokens")

## Section 8: Real DAO Governance Analysis

Let's analyze governance patterns from real DAOs using simulated data based on actual on-chain statistics.

In [ ]:
# Simulated real DAO data (based on actual on-chain data patterns)
real_dao_data = {
    'MakerDAO': {
        'token': 'MKR',
        'total_supply': 1_005_577,
        'top_10_pct': 48.2,
        'avg_participation': 12.5,
        'proposals_per_month': 8,
        'avg_approval_rate': 78.3,
        'timelock_hours': 48,
        'voter_turnout_trend': 'declining'
    },
    'Uniswap': {
        'token': 'UNI',
        'total_supply': 1_000_000_000,
        'top_10_pct': 52.1,
        'avg_participation': 8.3,
        'proposals_per_month': 3,
        'avg_approval_rate': 82.1,
        'timelock_hours': 168,  # 7 days
        'voter_turnout_trend': 'stable'
    },
    'Compound': {
        'token': 'COMP',
        'total_supply': 10_000_000,
        'top_10_pct': 57.8,
        'avg_participation': 15.2,
        'proposals_per_month': 4,
        'avg_approval_rate': 71.5,
        'timelock_hours': 48,
        'voter_turnout_trend': 'increasing'
    },
    'Aave': {
        'token': 'AAVE',
        'total_supply': 16_000_000,
        'top_10_pct': 42.3,
        'avg_participation': 11.8,
        'proposals_per_month': 6,
        'avg_approval_rate': 85.2,
        'timelock_hours': 24,
        'voter_turnout_trend': 'stable'
    },
    'Curve': {
        'token': 'CRV',
        'total_supply': 3_030_000_000,
        'top_10_pct': 61.5,
        'avg_participation': 22.4,  # Higher due to veCRV locking
        'proposals_per_month': 5,
        'avg_approval_rate': 88.7,
        'timelock_hours': 72,
        'voter_turnout_trend': 'increasing'
    }
}

# Create comparison DataFrame
df_daos = pd.DataFrame(real_dao_data).T
df_daos = df_daos.reset_index().rename(columns={'index': 'DAO'})

print("Real DAO Governance Comparison")
print("=" * 90)
print(df_daos[['DAO', 'token', 'top_10_pct', 'avg_participation', 'avg_approval_rate', 'timelock_hours']].to_string(index=False))

In [ ]:
# Visualize DAO comparisons
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

daos = df_daos['DAO'].tolist()
colors = ['#3498db', '#e74c3c', '#27ae60', '#9b59b6', '#f39c12']

# 1. Token Concentration
axes[0, 0].barh(daos, df_daos['top_10_pct'], color=colors)
axes[0, 0].set_xlabel('Top 10 Holders (% of Supply)')
axes[0, 0].set_title('Token Concentration by DAO')
axes[0, 0].axvline(x=50, color='red', linestyle='--', alpha=0.7, label='50% threshold')
for i, v in enumerate(df_daos['top_10_pct']):
    axes[0, 0].text(v + 1, i, f'{v:.1f}%', va='center')

# 2. Voter Participation
axes[0, 1].barh(daos, df_daos['avg_participation'], color=colors)
axes[0, 1].set_xlabel('Average Participation (%)')
axes[0, 1].set_title('Voter Participation by DAO')
for i, v in enumerate(df_daos['avg_participation']):
    axes[0, 1].text(v + 0.5, i, f'{v:.1f}%', va='center')

# 3. Approval Rates
axes[1, 0].barh(daos, df_daos['avg_approval_rate'], color=colors)
axes[1, 0].set_xlabel('Average Approval Rate (%)')
axes[1, 0].set_title('Proposal Approval Rates')
axes[1, 0].set_xlim(60, 100)
for i, v in enumerate(df_daos['avg_approval_rate']):
    axes[1, 0].text(v + 0.5, i, f'{v:.1f}%', va='center')

# 4. Timelock Duration
axes[1, 1].barh(daos, df_daos['timelock_hours'], color=colors)
axes[1, 1].set_xlabel('Timelock Duration (hours)')
axes[1, 1].set_title('Security Timelock by DAO')
for i, v in enumerate(df_daos['timelock_hours']):
    axes[1, 1].text(v + 2, i, f'{v}h', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Simulate historical voting patterns
def simulate_voting_history(dao_name: str, num_proposals: int = 20) -> pd.DataFrame:
    """Simulate voting history for a DAO."""
    dao_info = real_dao_data[dao_name]
    
    proposals = []
    base_date = datetime(2023, 1, 1)
    
    for i in range(num_proposals):
        # Participation varies
        participation = np.random.normal(dao_info['avg_participation'], 3)
        participation = max(3, min(40, participation))  # Bound between 3-40%
        
        # Approval varies
        approval = np.random.normal(dao_info['avg_approval_rate'], 10)
        approval = max(30, min(99, approval))
        
        # Some proposals fail
        passed = approval > 50 and participation > 4
        
        proposals.append({
            'proposal_id': i + 1,
            'date': base_date + timedelta(days=i * 15),
            'participation_pct': participation,
            'approval_pct': approval,
            'passed': passed,
            'votes_for': int(dao_info['total_supply'] * participation/100 * approval/100),
            'votes_against': int(dao_info['total_supply'] * participation/100 * (100-approval)/100)
        })
    
    return pd.DataFrame(proposals)


# Generate and display voting history
print("Simulated Voting History: Uniswap")
print("=" * 80)

uni_history = simulate_voting_history('Uniswap', 15)
print(uni_history.to_string(index=False))

# Summary stats
print(f"\nSummary:")
print(f"  Proposals passed: {uni_history['passed'].sum()}/{len(uni_history)}")
print(f"  Avg participation: {uni_history['participation_pct'].mean():.1f}%")
print(f"  Avg approval: {uni_history['approval_pct'].mean():.1f}%")

In [ ]:
# Visualize voting trends
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Participation over time
axes[0].plot(uni_history['proposal_id'], uni_history['participation_pct'], 'b-o', label='Participation')
axes[0].axhline(y=real_dao_data['Uniswap']['avg_participation'], color='r', linestyle='--', 
                label=f"Average ({real_dao_data['Uniswap']['avg_participation']}%)")
axes[0].fill_between(uni_history['proposal_id'], uni_history['participation_pct'], alpha=0.3)
axes[0].set_xlabel('Proposal #')
axes[0].set_ylabel('Participation (%)')
axes[0].set_title('Uniswap: Voter Participation Over Time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Right: Pass/Fail scatter
passed = uni_history[uni_history['passed']]
failed = uni_history[~uni_history['passed']]

axes[1].scatter(passed['participation_pct'], passed['approval_pct'], 
                c='green', s=100, label='Passed', alpha=0.7)
axes[1].scatter(failed['participation_pct'], failed['approval_pct'], 
                c='red', s=100, label='Failed', alpha=0.7, marker='x')
axes[1].axhline(y=50, color='gray', linestyle='--', alpha=0.5)
axes[1].axvline(x=4, color='gray', linestyle='--', alpha=0.5)  # Quorum line
axes[1].set_xlabel('Participation (%)')
axes[1].set_ylabel('Approval (%)')
axes[1].set_title('Proposal Outcomes')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("  1. Participation varies significantly between proposals")
print("  2. Most proposals pass with >70% approval")
print("  3. Failed proposals often have low participation or contentious votes")

## Section 9: Challenge Exercises

Test your understanding with these challenges!

### Challenge 1: Implement Conviction Voting

Conviction voting increases voting power the longer tokens are staked on a position. Implement a basic version.

In [ ]:
# YOUR TURN: Implement Conviction Voting

class ConvictionVoter:
    """
    Conviction voting: voting power grows over time.
    
    Formula: conviction = balance * (1 - decay^days_staked)
    Where decay is typically 0.9 (conviction grows to max over ~30 days)
    """
    
    def __init__(self, address: str, balance: float, decay: float = 0.9):
        self.address = address
        self.balance = balance
        self.decay = decay
        self.vote_start: Optional[datetime] = None
        self.current_vote: Optional[str] = None  # 'for' or 'against'
    
    def start_voting(self, position: str) -> None:
        """Start accumulating conviction for a position."""
        # TODO: Implement this
        # Set vote_start to now
        # Set current_vote to position
        pass
    
    def get_conviction(self) -> float:
        """
        Calculate current conviction based on time staked.
        
        Returns:
            Current conviction value
        """
        # TODO: Implement this
        # If not voting, return 0
        # Calculate days since vote_start
        # Apply formula: balance * (1 - decay^days)
        pass
    
    def change_vote(self, new_position: str) -> None:
        """Change vote position (resets conviction)."""
        # TODO: Implement this
        # Reset vote_start to now
        # Update current_vote
        pass


# Test your implementation
print("Challenge 1: Conviction Voting")
print("=" * 70)
print("\nImplement the ConvictionVoter class above!")
print("\nExpected behavior:")
print("  - Conviction starts at 0 when voting begins")
print("  - Grows asymptotically toward balance over ~30 days")
print("  - Resets when changing position")
print("\nBonus: Plot conviction growth over 30 days")

### Challenge 2: Detect Governance Attack

Write a function to detect suspicious governance activity.

In [ ]:
# YOUR TURN: Implement Attack Detection

def detect_governance_attack(dao: SimpleDAO, proposal: Proposal) -> Dict[str, any]:
    """
    Analyze a proposal vote for signs of attack.
    
    Red flags to detect:
    1. Single voter > 30% of votes
    2. Top 3 voters > 60% of votes  
    3. Unusual voting pattern (all one direction)
    4. Very low participation with high approval
    5. New addresses voting with large amounts
    
    Returns:
        Dictionary with risk assessment
    """
    warnings = []
    risk_score = 0  # 0-100
    
    # TODO: Implement detection logic
    # Analyze voting patterns
    # Check for concentration
    # Look for anomalies
    
    # Example structure:
    # if single_voter_dominance > 0.3:
    #     warnings.append("Single voter controls >30% of votes")
    #     risk_score += 30
    
    return {
        'risk_score': risk_score,
        'risk_level': 'LOW' if risk_score < 30 else 'MEDIUM' if risk_score < 60 else 'HIGH',
        'warnings': warnings
    }


print("Challenge 2: Attack Detection")
print("=" * 70)
print("\nImplement the detect_governance_attack function!")
print("\nTest cases to consider:")
print("  1. Normal vote with distributed voters")
print("  2. Whale-dominated vote")
print("  3. Suspiciously uniform voting")
print("  4. Flash loan attack pattern")

### Challenge 3: Design an Improved Governance System

Design a governance system that addresses the weaknesses we've identified.

In [ ]:
# YOUR TURN: Design Improved Governance

print("Challenge 3: Design Better Governance")
print("=" * 70)
print("""
Design a governance system that addresses:

1. WHALE DOMINANCE
   Current: 1 token = 1 vote
   Your solution: ?

2. LOW PARTICIPATION
   Current: ~10% average turnout
   Your solution: ?

3. FLASH LOAN ATTACKS
   Current: No protection
   Your solution: ?

4. VOTE BUYING
   Current: Easily done via DeFi
   Your solution: ?

Write your design below as comments or implement it!
""")

# Your design here:
# class ImprovedDAO:
#     """
#     Your improved DAO design.
#     """
#     pass

## Summary

In this notebook, you learned:

### Key Concepts

1. **DAO Structure**
   - Governance tokens represent voting power
   - Proposals go through creation, voting, and execution phases
   - Smart contracts enforce governance rules automatically

2. **Token-Weighted Voting**
   - Simple: 1 token = 1 vote
   - Leads to wealth concentration = power concentration
   - Most DAOs have highly concentrated token distributions

3. **Quadratic Voting**
   - Voting power = sqrt(tokens)
   - Reduces whale influence significantly
   - Vulnerable to Sybil attacks without identity verification

4. **Governance Attacks**
   - Whale dominance: Large holders can unilaterally pass proposals
   - Flash loan attacks: Borrow tokens, vote, return in one transaction
   - Vote buying: Bribing holders via DeFi protocols
   - Low participation exploitation: Small amounts can swing votes

5. **Delegation**
   - Allows passive participation through trusted delegates
   - Can further concentrate power in popular delegates
   - Liquid democracy enables flexible representation

6. **Security Mechanisms**
   - Time-locks: Delay execution to allow community response
   - Multi-sig: Require multiple approvals for actions
   - Guardians: Emergency cancellation powers

### Real-World Insights

- Average DAO participation is only 10-15%
- Top 10 holders typically control 40-60% of tokens
- Most proposals pass with >70% approval
- Governance is an unsolved problem in decentralization

### Further Reading

- [Vitalik on DAOs](https://vitalik.ca/general/2021/08/16/voting3.html) - Moving beyond token voting
- [Compound Governance](https://compound.finance/docs/governance) - Real implementation
- [Quadratic Voting Explained](https://www.radicalxchange.org/concepts/quadratic-voting/) - RadicalxChange
- [Conviction Voting](https://medium.com/giveth/conviction-voting-a-novel-continuous-decision-making-alternative-to-governance-aa746cfb9475) - Alternative mechanism